In [ ]:
# import packages
import pandas as pd

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

from column_checks import get_dtypes
from filepaths import *
import data_cleaning

year = 2020

# Functions for loading intermediate outputs

In [ ]:
# load data from csv
year = 2020
path_prefix = f"{year}/"

cems = pd.read_csv(outputs_folder(f"{path_prefix}/cems_subplant_{year}.csv"), dtype=get_dtypes(), parse_dates=['datetime_utc', 'report_date'])
partial_cems_plant = pd.read_csv(outputs_folder(f"{path_prefix}/partial_cems_plant_{year}.csv"), dtype=get_dtypes(), parse_dates=['datetime_utc', 'report_date'])
partial_cems_subplant = pd.read_csv(outputs_folder(f"{path_prefix}/partial_cems_subplant_{year}.csv"), dtype=get_dtypes(), parse_dates=['datetime_utc', 'report_date'])
eia923_allocated = pd.read_csv(outputs_folder(f"{path_prefix}/eia923_allocated_{year}.csv"), dtype=get_dtypes(), parse_dates=['report_date'])
plant_attributes = pd.read_csv(outputs_folder(f"{path_prefix}/plant_static_attributes_{year}.csv"), dtype=get_dtypes())

In [ ]:
# load data from csv
year = 2020
path_prefix = f"{year}/"

eia923_allocated, primary_fuel_table = data_cleaning.clean_eia923(year, False)
cems = data_cleaning.clean_cems(year, False, primary_fuel_table)

In [ ]:
data = cems[cems["plant_id_eia"] == 673]

data.groupby(["plant_id_eia","unitid","report_date"]).sum()

In [ ]:
eia923_allocated[eia923_allocated["plant_id_eia"] == 50949].to_csv("test.csv")

## Explore Results

In [ ]:
# load all power sector results and concat together

resolution = "annual"

all_data = []
for ba in os.listdir(results_folder(f"2020/power_sector_data/{resolution}/us_units")):
    df = pd.read_csv(results_folder(f"2020/power_sector_data/{resolution}/us_units/{ba}"))
    df["ba_code"] = ba.split(".")[0]
    all_data.append(df)

all_data = pd.concat(all_data, axis=0)

all_data[all_data["fuel_category"] == "total"]

## Explore the difference between adjusted and unadjusted (for biomass) factors

In [ ]:
all_data["pctdiff"] = (all_data.generated_co2_rate_lb_per_mwh_for_electricity_adjusted - all_data.generated_co2_rate_lb_per_mwh_for_electricity) / all_data.generated_co2_rate_lb_per_mwh_for_electricity


all_data.loc[all_data["fuel_category"] == "total", ["ba_code","pctdiff","generated_co2_rate_lb_per_mwh_for_electricity_adjusted","generated_co2_rate_lb_per_mwh_for_electricity"]].sort_values(by="pctdiff").head(25)